# Whereabout Streets Data Extraction
This notebook will demonstrate how to access Street and Bridge Operations PDF file and extract this data to create a work order plan template.

<div style="text-align:center"><img src="https://upload.wikimedia.org/wikipedia/en/9/94/Closeup_of_pavement_with_grass.JPG" /></div>

## Introduction
The purpose of this notebook is to create a Street and Bridge Work Order plans based on segment IDs and additional comments on long line. Markings feature layers are published in the City of Austin ArcGIS Portal page available for public view as well. 

The schedule for where sealcoat and overlay streets are completed is received through email by Street and Bridge Operations on a daily basis. It is sent as a PDF file that lists weather conditions, temperature, and provides a table of streets where paving is completed.

<b>The only manual process the user will have to do is to:</b>
- Input Segment IDs
- Make comments on long line markings
- Specify file path to retrieve the table of completed streets paved for PDF name and file path
- Create any missing markings assets that are not visible in aerial imagery

This process will cut down on the previous process of manually editing a plans layout through copy-pasting imagery and writing Location IDs, work groups, markings found, and the exporting plans one at a time. An excel document will be created based on this input and read segment IDs to find all short line and specialty point markings. This will ideally generate multiple PDF plans in a faster and shorter time frame.

<i><b>Disclaimer:</b> This product is for informational purposes and may not have been prepared for or be suitable for legal, engineering, or surveying purposes. No warranty is made by the City of Austin regarding specific accuracy or completeness.</i>

## Imports
The packages used for this project are:
- [pandas](https://pandas.pydata.org/) to create dataframe of extracted table and transform the data
- [openpyxl](https://openpyxl.readthedocs.io/en/stable/) to edit excel files
- [arcgis](https://esri.github.io/arcgis-python-api/apidoc/html/) to search for markings feature layer dataset

In [1]:
import pandas as pd
%run C:\Users\Govs\Projects/CopyGISFeatures.py
%run C:\Users\Govs\Projects/FeatureLayerDataFrame.py
    
from openpyxl import Workbook,load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows

from arcgis.gis import GIS
from arcgis.features import FeatureLayer

from functools import reduce
import numpy as np

## Constants

The date by month and day constant will determine the file pdf name to use as a dataframe. Folder path will determine where the plans will be created depending on the year. This is set to the top for the purpose of changing these constants as needed.

<i>The table below explains the purpose of each constant.</i>

| Constant | Description   |
|:--------:|----|
| <b>MONTH, DAY, YEAR</b> |Date used to find PDF in month-day format and file path based on year|
|<b>FOLDER</b>      |File directory used to import SBO whereabouts reports from email|
|<b>FILE_NAME</b>   |File directory name used to extact SBO whereabouts reports from file|
|<b>SIGN_IN</b>   |Whether to prompt user to sign in to outlook email|
|<b>INPUT</b>|Whether to prompt user to input segment Ids and comments to export to excel| 

In [2]:
YEAR = str(2020)
FOLDER = (r"G:\ATD\Signs_and_Markings\MARKINGS\Whereabouts WORK ORDERS\{}"
         ).format(YEAR)
FILE_NAME = FOLDER + r'\08_August\sbo'
EXCEL_FILE = FILE_NAME + '.xlsx'
CSV_FILE = FILE_NAME + ".csv"
INPUT= True
%store FOLDER
%store EXCEL_FILE

Stored 'FOLDER' (str)
Stored 'EXCEL_FILE' (str)


## Methods
These functions will be used to extract and transform the data into a feasible format.

<i>The table below explains the purpose of each:</i>

| Method | Description   |
|:--------:|----|
|<b>input_form</b> |Prompts user to input segment IDs and long line specifications|
|<b>query_df</b>   |Query dataframe by segment IDs|

In [3]:
# Prompts user to input segment IDs and longline while changing the datafram to include user input
def input_form(df):
    segments = []
    longline = []
    for index,row in df.iterrows():
        console = input(row['Location'] + "\nSegment ID list: ")
        if not console:
            segments.append(None)
            longline.append(None)
        else:
            temp = GISFeatures(console).to_df()
            segments.append(str(list(temp.SEGMENT_ID))[1:-1])
            comment = input("Longline: ")
            longline.append(comment)
    df['Segment IDs'], df['LongLine'] = (segments,longline)
    print("\nInput complete.")
    return df.dropna(subset=['Segment IDs'])
    
# Returns query dataframe appended if markings exist in the listed segment IDs
def query_df(fc,index,f,df,df1):
    q = "SEGMENT_ID IN('{})".format(df["Segment IDs"][index])
    if q != "SEGMENT_ID IN(N/A)":
        c = fc.query(where=q,return_count_only=True) 
        if c != 0:
            sdf = fc.query(where=q).sdf.filter(items=f)
            sdf["Location ID"] = df["Location ID"][index]
            sdf["LongLine"] = df["LongLine"][index]
            df1 = df1.append(sdf,sort=True)
    df1['COUNTS'] = 1
    return df1

# Return dataframe of the listed specifications
def specifications(df,i):
    df["SPECIFICATIONS"] = ''
    for index,row in df.iterrows():
        keys = list(row[i:])
        values = list(df.columns)[i:]
        spec = []
        for k,v in zip(keys,values):
            if k != 'N/A' and k != '' and v != 'WORK GROUPS':
                spec.append('{} {}'.format(int(k),v.lower().replace('_',' ')))
            if row['LongLine'] != 'N/A':
                sentence = 'Install {}, '.format(row['LongLine']) + ', '.join(word for word in spec)
            else:
                sentence = 'Install ' + ', '.join(word for word in spec)
        df.at[index,'SPECIFICATIONS'] = sentence
    if 'WORK GROUPS' in df.columns:
        df.loc[df.Street != None,'WORK GROUPS'] = df.loc[df.Street != None,'WORK GROUPS'].apply(str)
    return df

# Returns dataframe of markings count and pages
def location_in_df(df,markings_type,workgroup):
    if 'Location ID' in df:
        count = df.groupby(['Location ID',markings_type]).count()[['SEGMENT_ID']].rename(columns={"SEGMENT_ID":'COUNTS'})
        count = count.pivot_table(values='COUNTS',index='Location ID',columns=(markings_type),aggfunc='first').reset_index()
        count[workgroup] = workgroup
        page = df.groupby(['Location ID','SEGMENT_ID','LongLine',markings_type]).count()[['COUNTS']]
        page = page.pivot_table(
            values='COUNTS',index=['Location ID','SEGMENT_ID','LongLine'],columns=(markings_type),aggfunc='first')
        return count,page

# Returns dataframe of cover page
def create_cover(cover,sl_count,sp_count,wg):
    cover.loc[cover.LongLine != 'N/A', wg[2]] = wg[2]
    cover.loc[cover.LongLine == 'N/A', wg[2]] = 'N/A' 
    if not sl_count.empty and not sp_count.empty:
        cover = reduce(lambda z,y: pd.merge_ordered(z,y,on='Location ID'), [cover,sl_count,sp_count])
    elif not sl_count.empty or not sp_count.empty:
        count = sl_count if sp_count.empty else sp_count
        wg_remove = 'SPECIALTY MARKINGS' if sp_count.empty else 'SHORTLINE'
        cover = pd.merge_ordered(count,sl_count,on='Location ID')
        wg.remove(wg_remove)
    else:
        cover = specifications(cover,9)
        return cover
    cover = cover.dropna(how='all',subset=list(cover.columns)[6:]).fillna('N/A')
    cover['WORK GROUPS'] = cover[wg].apply(','.join,1).apply(lambda x: [s for s in x.split(',') if s != 'N/A'])
    cover = cover.drop(columns = wg).fillna('N/A')
    cover = specifications(cover,9)
    cover['PAGE'] = 1
    return cover[cover['WORK GROUPS'] != '[]']

# Returns dataframe of pages
def create_pages(pages,sl_page,sp_page):
    if not sl_page.empty and not sp_page.empty: # if it has shortline and specialty
        pages = pd.merge_ordered(sl_page,sp_page,on=('Location ID','SEGMENT_ID','LongLine')).fillna("N/A")
        pages = specifications(pages,3)
        pages = pd.merge_ordered(pages,streets,on=('Location ID','SEGMENT_ID','LongLine'))
        pages = pages.sort_values(by=['Location ID','BLOCK']).drop(columns='BLOCK').reset_index(drop = True)
    elif not sl_page.empty or not sp_page.empty: # if it has shortline or specialty
        page = sl_page if sp_page.empty else sp_page
        pages = specifications(page.fillna('N/A'),3)
        pages = pd.merge_ordered(pages,streets,on=('Location ID','SEGMENT_ID','LongLine')).sort_values(
            by=['BLOCK','Location ID']).reset_index(drop = True).drop(columns='BLOCK')
        pages = pages.dropna(subset=['SPECIFICATIONS'])
        page = 1
        for index, row in streets.iterrows():
            if index != 0 and (row['Location ID'] != pages['Location ID'][index - 1]):
                page = 2
                pages.at[index,'PAGE'] = page
            else:
                page += 1
                pages.at[index,'PAGE'] = page
    else:
        pages.loc[cover.Street != None,'PAGE'] = 2
    return pages

# Creates worksheet in excel file unless the worksheet already exists
def create_ws(df,sheet_name):
    if sheet_name in wb:
        del wb[sheet_name]
    ws = wb.create_sheet(sheet_name)
    for r in dataframe_to_rows(df, index=False, header=True):
        ws.append(r)
    wb.save(EXCEL_FILE)

## Import SBO Report
The first thing to do is to import the sbo report from a csv file and convert it into a dataframe

In [ ]:
cols = {'wonum':'Location ID','on_street':'Street','from_street':'From','to_street':'To','actfinish_1':'Finish Date'}

sbo = pd.read_csv(CSV_FILE)

if 'wonum' in sbo.columns:
    df = sbo.filter(items=list(cols.keys())).sort_values('wonum').rename(columns=cols).reset_index(drop=True)
    df['Location'] = (df["Street"] + ' FROM ' + df["From"] + ' TO ' + df["To"]).str.upper()
    df.to_csv(CSV_FILE)
else:
    df = sbo.copy().drop(columns='Unnamed: 0').dropna(subset=['Segment IDs'])

This will display the first 10 rows of the report from SBO

In [4]:
df = pd.read_excel(EXCEL_FILE,index_col=0)
display(df.head(10))

,Location ID,Street,From,To,Type,Finish Date,Location,Segment IDs,LongLine
0,SG-54690,Meinardus Dr,St Elmo Rd,Sponberg Dr,Overlay,NaN,Meinardus Dr FROM St Elmo Rd TO Sponberg Dr,2023206,"double yellow centerline, white lane lines"


## Loading and Transforming Data

### PDF tables to Excel

Now that the PDFs have been extracted and exported to the folder path, the next step is to extract the tables in the PDF and export it as an excel file.

An input form will generate so the user can input Segment ID and comment information for each of the streets listed. The columns list will only take the relevant columns from the extracted table. The `pdfplumber` package will be used to extract tables from the PDF and prompt user to submit data.

The input will be stored as a DataFrame saved to an excel document. If the user already provided input froma  previous session, the dataframe will be set to the excel file document instead.

In [5]:
from pathlib import Path

# Will prompt input and export to excel unless the excel file already exists. In that case it will read excel file instead
if Path(EXCEL_FILE).exists():
    df.to_csv(CSV_FILE)
    df = pd.read_csv(CSV_FILE,index_col=0)
    df = df.reset_index(drop=True).fillna("N/A")
else:
    input_form(df)
    df = df.fillna("N/A")
    df['LongLine'] = ['N/A' if x == '' else x for x in df['LongLine']]
    df.to_excel(EXCEL_FILE)
    df.to_csv(CSV_FILE)

In [6]:
display(df)

,Location ID,Street,From,To,Type,Finish Date,Location,Segment IDs,LongLine
0,SG-54690,Meinardus Dr,St Elmo Rd,Sponberg Dr,Overlay,N/A,Meinardus Dr FROM St Elmo Rd TO Sponberg Dr,2023206,"double yellow centerline, white lane lines"


This file contains a table for the list of streets with the following columns:
- <i>Location ID</i>: unique identifier used for street paving
- <i>Street</i>: main street that is paved
- <i>From</i>: intersecting cross street
- <i>To</i>: intersecting cross street
- <i>Segment IDs</i>: list of segment IDs where street is paved seperated by commas
- <i>Comments</i>: Notes on long line markings

### Feature Layer Data Query

The next task is to find the markings through the list of segment IDs the user has inputted. For this task the `arcgis` package will be useful for extracting the markings available in each segment ID since the dataset is already available publically.

Since the markings datasets are publically available, we can login to ArcGIS Online anonymously. 

Use `client_id` instead of `None` if you wish to log-in through an AGOL federate account. Note that it will prompt user to enter code which can be found by following the instructions. Going through an AGOL federated account is useful if the user wishes to add their own layers as a reference such as [NearMap](https://go.nearmap.com/) aerial imagery. 

It will search through the markings feature layer based on the list of segment IDs provided by the excel file.

In [7]:
# variables used to find and query feature layer in AGOL
gis = GIS("https://austin.maps.arcgis.com/home/index.html")
url = r"https://services.arcgis.com/0L95CJ0VTaxqcmED/arcgis/rest/services/TRANSPORTATION_{}/FeatureServer/0"
sl,sp,streets = (pd.DataFrame(),pd.DataFrame(),pd.DataFrame())

# Columns for data frame. Indexes: df (0), shortline (1-4), specialty point (3 to etc.)
cols = ['SHORT_LINE_TYPE','SEGMENT_ID','SPECIALTY_POINT_TYPE','SPECIALTY_POINT_SUB_TYPE']
s_col = ['LEFT_BLOCK_FROM','RIGHT_BLOCK_FROM','SEGMENT_ID']

for index,row in df.iterrows():
    streets = query_df(FeatureLayer(url.format("street_segment")),index,s_col,df,streets)      
    sl = query_df(FeatureLayer(url.format("markings_short_line")),index,cols[:2],df,sl)
    sp = query_df(FeatureLayer(url.format("markings_specialty_point")),index,cols[1:],df,sp)
sp = FeatureLayerDataFrame('markings_specialty_point').specialty_markings(sp)

# Order table
streets['BLOCK'] = np.maximum(streets[s_col[0]],streets[s_col[1]])
streets = streets.sort_values(by=['BLOCK','Location ID']).reset_index(drop = True)
streets = streets.rename(columns={'COUNTS':'PAGE'}).drop(s_col[:2],axis=1)

page = 1
for index, row in streets.iterrows():
    if index != 0 and (row['Location ID'] != streets['Location ID'][index - 1]):
        page = 2
        streets.at[index,'PAGE'] = page
    else:
        page += 1
        streets.at[index,'PAGE'] = page


'where' parameter is invalid


RuntimeError: 
'where' parameter is invalid
(Error Code: 400)

### Plans Table Creation

In [8]:
display(sl_page)

NameError: name 'sl_page' is not defined

#### Cover Table

In [52]:
wg = ['SHORT LINE','SPECIALTY MARKINGS','LONGLINE']
sl_count,sl_page = location_in_df(sl,'SHORT_LINE_TYPE',wg[0])
display(sl_count)
#sp_count, sp_page = location_in_df(sp,'SPECIALTY_POINT_TYPE',wg[1])
cover = create_cover(df.copy(),sl_count,sp_count,wg)
pages = create_pages(df.copy(),sl_page,sp_page)

SHORT_LINE_TYPE,Location ID,CROSSWALK,STOP_LINE,SHORT LINE
0,SG-54661,2,2,SHORT LINE


KeyError: "['SPECIALTY MARKINGS', 'SHORT LINE'] not in index"

#### Cover Table
This dataframe lists pavement markings queried by segment IDs with the following columns:
- <i>Location ID</i>: Unique identifier used for street paving
- <i>Location</i>: Location of wherreabouts work
- <i>WORK GROUPS</i>: Type of markings work group assigned to work order
- <i>SPECIFICATIONS</i>: Lists all markings that need to be installed on work order.

The dataframe will be saves in an excel sheet for it to be used again to generate the template.

In [13]:
display(cover.filter(['Location ID','Location','Comments','WORK GROUPS','SPECIFICATIONS']))

,Location ID,Location,WORK GROUPS,SPECIFICATIONS
0,70194,Rainey St FROM Cummings St TO Driskill St,"['SHORT LINE', 'SPECIALTY MARKINGS', 'LONGLINE']","Install 1 yield line, 3 parking 'l'"
1,79816,River St FROM Red River St TO East Ave,"['SHORT LINE', 'SPECIALTY MARKINGS', 'LONGLINE']","Install 1 crosswalk, 2 parking 'l' , 2 parking..."
2,SG-98745935,Radam Ln FROM S 1st St TO James Casey,"['SHORT LINE', 'SPECIALTY MARKINGS', 'LONGLINE']","Install partial double yellow centerline, turn..."


#### Pages Table
This dataframe lists pavement markings queried by segment IDs with the following columns:
- <i>LOCATION ID</i>: Unique identifier used for street paving
- <i>SEGMENT_ID</i>: Segment ID of page reference
- <i>SPECIFICATIONS</i>: Lists all markings that need to be installed on work order.
- <i>PAGE</i>: Page number used for work order

The dataframe will be saves in an excel sheet for it to be used again to generate the template.

In [14]:
display(pages.filter(['Location ID','SEGMENT_ID','SPECIFICATIONS','PAGE'])) 

,Location ID,SEGMENT_ID,SPECIFICATIONS,PAGE
0,70194,2019466,NaN,2
1,79816,2019341,NaN,2
2,79816,2019355,NaN,2
3,79816,2019370,NaN,3
4,SG-98745935,2024315,"Install partial double yellow centerline, turn...",2


## Create Worksheets of DataFrames

In [15]:
wb = load_workbook(filename = EXCEL_FILE)
create_ws(cover,'Cover')
create_ws(pages,'Pages')

## Generating Whereabouts Plans
To generate whereabout plans, we will have to use the `arcpy` package, which requires Python 2 and ArcMap 10.5. Eventually, this notebook will be able to use `arcpy` in Python 3.

[Click here to access notebook](PlansTemplate.ipynb)